# Benchmark!
Compare our results with other state-of-the-art methods.

In [1]:
import re
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, HTML
ix = pd.IndexSlice

In [2]:
methods = pd.CategoricalDtype(categories=[
    'ProQ2D', 
    'ProQ3', 
    'ProQ3D', 
    'ProQ4', 
    'VoroMQA', 
    'RWplus', 
    '3D CNN', 
    'Ornate', 
    'AngularQA', 
    'GraphQA-RAW',
    'GraphQA',
], ordered=True)
tests = pd.CategoricalDtype(categories=[
    '3DRobot', 
    'CASP 5',
    'CASP 6',
    'CASP 7',
    'CASP 8',
    'CASP 9',
    'CASP 10', 
    'CASP 10 stage 1', 
    'CASP 10 stage 2', 
    'CASP 11', 
    'CASP 11 stage 1', 
    'CASP 11 stage 2', 
    'CASP 12', 
    'CASP 12 stage 1', 
    'CASP 12 stage 2', 
    'CASP 13', 
    'CAMEO'
], ordered=True)

## Read all metrics

In [3]:
df = pd.concat((
    pd.read_csv('other_results.csv', comment='#'),
    pd.read_csv('our_results.csv', comment='#')
), axis='index', ignore_index=True, sort=True)
df['Method'] = df['Method'].astype(methods)
df['TestSet'] = df['TestSet'].astype(tests)
df.pivot_table(index=['TestSet', 'Method', 'TrainValSets', 'AsReportedBy'], columns=['Output', 'Metric'], values=['Value']) \
    .droplevel(0, axis='columns') \
    .style \
    .applymap(lambda v: 'opacity:0' if np.isnan(v) else '')

Use this with tables generator for a specific CASP dataset

In [4]:
dataset ='CASP 13'

df = pd.concat((
    pd.read_csv('other_results.csv', comment='#'),
    pd.read_csv('our_results.csv', comment='#')
), axis='index', ignore_index=True, sort=True)
df['Method'] = df['Method'].astype(methods)
df['TestSet'] = df['TestSet'].astype(tests)
display(
    df[(df.Output != 'Global LDDT') & (df.Method != 'ProQ2D') & (df.Method != 'ProQ3') & (df.TestSet.str.startswith(dataset) | df.TestSet.str.startswith(dataset))]
    .pivot_table(index=['TestSet', 'Method', 'TrainValSets', 'AsReportedBy'], columns=['Output', 'Metric'], values=['Value'])
    .droplevel(0, axis='columns')
    .sort_index()
    .round(3)
    .style
    .applymap(lambda v: 'opacity:0' if np.isnan(v) else '')
)

# To import into tablesgenerator
print(
    df[(df.Output != 'Global LDDT') & (df.Method != 'ProQ2D') & (df.Method != 'ProQ3') & (df.TestSet.str.startswith(dataset) | df.TestSet.str.startswith(dataset))]
    .pivot_table(index=['TestSet', 'Method', 'TrainValSets', 'AsReportedBy'], columns=['Output', 'Metric'], values=['Value'])
    .droplevel(0, axis='columns')
    .sort_index()
    .round(3)
    .to_csv()
          .replace('Global GDT_TS', '\\gdtts').replace('Local LDDT', '\\lddt').replace('Output','').replace('Metric','')
          .replace('First Rank Loss', 'FRL')
          .replace('R per target', r'$R_\text{target}$')
          .replace('R per model', r'$R_\text{model}$')
          .replace('ρ per target', r'$\rho_\text{target}$')
          .replace('ρ per model', r'$\rho_\text{model}$')
          .replace('τ per target', r'$\tau_\text{target}$')
          .replace('τ per model', r'$\tau_\text{model}$')
          .replace('ρ', r'$\rho$')
          .replace('τ', r'$\tau$')
          .replace('GraphQA-RAW', r'$\graphqaeq_\textsc{Raw}$')
          .replace('GraphQA', r'\graphqa')
          .replace('AsReportedBy', 'Source')
         )

,,,,\gdtts,\gdtts,\gdtts,\gdtts,\gdtts,\gdtts,\gdtts,\gdtts,\lddt,\lddt,\lddt
,,,,FRL,R,$R_\text{target}$,RMSE,$\rho$,$\rho_\text{target}$,$\tau$,$\tau_\text{target}$,R,$R_\text{model}$,RMSE
TestSet,Method,TrainValSets,Source,,,,,,,,,,,
CASP 13,ProQ3D,CASP 10+11+12,Ours,0.16,0.849,0.671,0.129,0.81,0.619,0.625,0.458,,,
CASP 13,ProQ4,CASP 10+11+12,Ours,0.147,0.671,0.733,0.182,0.645,0.668,0.492,0.508,,,
CASP 13,VoroMQA,PDB,Ours,0.024,0.767,0.665,0.177,0.765,0.606,0.571,0.443,,,
CASP 13,3D CNN,CASP 7+8+9+10+11+12,Ours,0.135,0.661,0.753,0.192,0.632,0.662,0.457,0.487,,,
CASP 13,Ornate,CASP 7+8+9+10+11+12,Ours,0.283,0.533,0.646,0.352,0.522,0.642,0.36,0.467,,,
CASP 13,$\graphqaeq_\textsc{Raw}$,CASP 7+8+9+10,Ours,0.122,0.767,0.727,0.167,0.774,0.701,0.573,0.524,0.723,0.475,0.154
CASP 13,\graphqa,CASP 7+8+9+10,Ours,0.093,0.846,0.834,0.123,0.84,0.799,0.647,0.616,0.764,0.539,0.142



## Group by dataset and show all metrics

In [5]:
def style_metrics_groupby(subgroup):
    if subgroup.columns.get_level_values('Metric')[0].startswith('First') or subgroup.columns.get_level_values('Metric')[0] == 'RMSE':
        color = 'DodgerBlue'
        m = np.nanmin(subgroup.values)
    else:
        color = 'green'
        m = np.nanmax(subgroup.values)
    style = np.where(subgroup == m, f'color: {color}; font-weight: bold;', '')
    border = np.full(style.shape, False)
    border[:, -1] = True
    border = np.where(border, 'border-right: 1px solid gray;', '')
    css = np.char.add(style, border)
    return pd.DataFrame(css, index=subgroup.index, columns=subgroup.columns)

In [6]:
columns = ['TestSet']

for group_keys, group in df.groupby(columns):
    if isinstance(group_keys, str):
        group_keys = (group_keys,)
    group = (
        group
            .drop(columns=columns)
            .pivot_table(index=['Method', 'TrainValSets'], columns=['Output', 'Metric', 'AsReportedBy'], values=['Value'])
            .droplevel(0, axis='columns')
    )
    styled = (
        group.style
        .set_caption('<h3 align="left">' + ' - '.join(group_keys) + '</h3>')
        .applymap(lambda v: 'opacity:.1' if np.isnan(v) else '')
        .apply(lambda g: g.groupby(axis='columns', level=['Output', 'Metric']).apply(style_metrics_groupby), axis=None)
        .format(lambda v: f'{v:.3f}' if np.isfinite(v) else '')
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
    )
    display(styled)

In [7]:
columns = ['TestSet']

for group_keys, group in df[df.Output != 'Global LDDT'].groupby(columns):
    if isinstance(group_keys, str):
        group_keys = (group_keys,)
    group = (
        group
            .drop(columns=columns)
            .pivot_table(index=['Method', 'TrainValSets'], columns=['Output', 'Metric', 'AsReportedBy'], values=['Value'])
            .droplevel(0, axis='columns')
    )
    styled = (
        group.style
        .set_caption('<h3 align="left">' + ' - '.join(group_keys) + '</h3>')
        .applymap(lambda v: 'opacity:.1' if np.isnan(v) else '')
        .apply(lambda g: g.groupby(axis='columns', level=['Output', 'Metric']).apply(style_metrics_groupby), axis=None)
        .format(lambda v: f'{v:.3f}' if np.isfinite(v) else '')
        .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
    )
    display(styled)
    
    # To import into tablesgenerator
    # print(group.round(3).to_csv()
    #       .replace('Global GDT_TS', '\\gdtts').replace('Local LDDT', '\\lddt').replace('Output','').replace('Metric','')
    #       .replace('First Rank Loss', 'FRL')
    #       .replace('R per target', r'$R_\text{target}$')
    #       .replace('R per model', r'$R_\text{model}$')
    #       .replace('ρ per target', r'$\rho_\text{target}$')
    #       .replace('ρ per model', r'$\rho_\text{model}$')
    #       .replace('τ per target', r'$\tau_\text{target}$')
    #       .replace('τ per model', r'$\tau_\text{model}$')
    #       .replace('GraphQA-RES', r'$\graphqaeq_\textsc{Raw}$')
    #       .replace('GraphQA', r'\graphqa')
    #      )

## Summary for paper
- Global LDDT ignored
- Stage 1 removed
- Stage 2 treated as the whole dataset
- If multiple values are reported for the same (method, dataset) keep the best

In [8]:
df = pd.concat((
    pd.read_csv('other_results.csv', comment='#'),
    pd.read_csv('our_results.csv', comment='#')
), axis='index', ignore_index=True, sort=True)

# Remove Global LDDT
df = df.query('Output != "Global LDDT"')
df['Output'] = df['Output'].str.replace(re.compile('(Local|Global) '), '')

# Remove stage 1
df = df[~df['TestSet'].str.endswith("stage 1")]

# Treat stage 2 as the whole dataset
df['TestSet'] = df['TestSet'].str.replace(' stage 2', '').astype(tests)
df = df.drop('TrainValSets', axis='columns')

df['Method'] = df['Method'].astype(methods)

# Find best across AsReportedBy (FRL and RMSE need to be negated and negated back)
df.loc[df['Metric'] == 'First Rank Loss', 'Value'] *= -1
df.loc[df['Metric'] == 'RMSE', 'Value'] *= -1
df = df.pivot_table(values='Value', index=['TestSet', 'Method'], columns=['Output', 'Metric'], aggfunc='max')
df.loc[:, df.columns.get_loc_level('First Rank Loss', level='Metric')[0]] *= -1
df.loc[:, df.columns.get_loc_level('RMSE', level='Metric')[0]] *= -1

In [9]:
def highlight_min_group(series):
    def highlight_min(group):
        css = pd.Series([''] * len(group), index=group.index)
        if np.isfinite(group).any():
            css.loc[group.idxmin()] += 'color: DodgerBlue; font-weight: bold;'
        css.iloc[-1] += 'border-bottom: 2px solid gray;'
        return css
    return series.groupby('TestSet').apply(highlight_min)

def highlight_max_group(series):
    def highlight_max(group):
        css = pd.Series([''] * len(group), index=group.index)
        if np.isfinite(group).any():
            css.loc[group.idxmax()] += 'color: green; font-weight: bold;'
        css.iloc[-1] += 'border-bottom: 2px solid gray;'
        return css
    return series.groupby('TestSet').apply(highlight_max)

In [10]:
the_higher_the_better = [c for c in df.columns if c[1] not in {'First Rank Loss', 'RMSE'}]
the_lower_the_better =  [c for c in df.columns if c[1] in {'First Rank Loss', 'RMSE'}]

df.style \
    .format(lambda v: f'{v:.3f}' if np.isfinite(v) else '') \
    .apply(highlight_min_group, subset=the_lower_the_better, axis='index') \
    .apply(highlight_max_group, subset=the_higher_the_better, axis='index') \
    .set_precision(3) \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])

In [11]:
df_simple = df \
    .drop(index=['3DRobot'], level='TestSet') \
    .drop(index=['ProQ2D', 'ProQ3'], level='Method') \
    .loc[:, [c[1] in {'First Rank Loss', 'RMSE', 'R', 'R per target', 'R per model'} for c in df.columns]]

the_higher_the_better = [c for c in df_simple.columns if c[1] not in {'First Rank Loss', 'RMSE'}]
the_lower_the_better =  [c for c in df_simple.columns if c[1] in {'First Rank Loss', 'RMSE'}]

df_simple.style \
    .format(lambda v: f'{v:.4f}' if np.isfinite(v) else '') \
    .apply(highlight_min_group, subset=the_lower_the_better, axis='index') \
    .apply(highlight_max_group, subset=the_higher_the_better, axis='index') \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])